<a href="https://colab.research.google.com/github/kauecapellato/artigo_ic/blob/main/BaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing the historic data of games 

In [1]:
!pip install selenium

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, log_loss, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from seaborn import heatmap
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
path = "/content/gdrive/MyDrive/csv_files/"

columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A','BWH', 'BWD', 'BWA', 
                'IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA']

#'BbMx>2.5',	'BbAv>2.5',	'BbMx<2.5',	'BbAv<2.5',	'BbAH',	'BbAHh',	'BbMxAHH',	'BbAvAHH',	'BbMxAHA',	'BbAvAHA']


#playing_statistics_1 = pd.read_csv(path + 'E0.csv', usecols = columns_req, nrows = 381)
#playing_statistics_2 = pd.read_csv(path + 'E0 (1).csv', usecols = columns_req, nrows = 381)
#playing_statistics_3 = pd.read_csv(path + 'E0 (2).csv', usecols = columns_req, nrows = 381)
#playing_statistics_4 = pd.read_csv(path + 'E0 (3).csv', usecols = columns_req, nrows = 381)
#playing_statistics_5 = pd.read_csv(path + 'E0 (4).csv', usecols = columns_req, nrows = 381)
#playing_statistics_6 = pd.read_csv(path + 'E0 (5).csv', usecols = columns_req, nrows = 381)
#playing_statistics_7 = pd.read_csv(path + 'E0 (6).csv', usecols = columns_req, nrows = 381)
#playing_statistics_8 = pd.read_csv(path + 'E0 (7).csv', usecols = columns_req, nrows = 381)
#playing_statistics_9 = pd.read_csv(path + 'E0 (8).csv', usecols = columns_req, nrows = 381)
#playing_statistics_10 = pd.read_csv(path + 'E0 (9).csv', usecols = columns_req, nrows = 381)
playing_statistics_11 = pd.read_csv(path + 'E0 (10).csv', usecols = columns_req, nrows = 381)
playing_statistics_12 = pd.read_csv(path + 'E0 (11).csv', usecols = columns_req, nrows = 381)
playing_statistics_13 = pd.read_csv(path + 'E0 (12).csv', usecols = columns_req, nrows = 381)
playing_statistics_14 = pd.read_csv(path + 'E0 (13).csv', usecols = columns_req, nrows = 381)
playing_statistics_15 = pd.read_csv(path + 'E0 (14).csv', usecols = columns_req, nrows = 381)
playing_statistics_16 = pd.read_csv(path + 'E0 (15).csv', usecols = columns_req, nrows = 381)
playing_statistics_17 = pd.read_csv(path + 'E0 (16).csv', usecols = columns_req, nrows = 381)
playing_statistics_18 = pd.read_csv(path + 'E0 (17).csv', usecols = columns_req, nrows = 381)
playing_statistics_19 = pd.read_csv(path + 'E0 (18).csv', usecols = columns_req, nrows = 381)
playing_statistics_20 = pd.read_csv(path + 'E0 (19).csv', usecols = columns_req, nrows = 381)

In [5]:
# example
playing_statistics_11.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA
0,14/08/10,Aston Villa,West Ham,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,1.91,3.25,4.33,2.00,3.25,4.20
1,14/08/10,Blackburn,Everton,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,2.88,3.20,2.50,3.10,3.30,2.38
2,14/08/10,Bolton,Fulham,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.4,2.15,3.25,3.25,2.20,3.2,3.0,2.15,3.20,3.50,2.20,3.30,3.40
3,14/08/10,Chelsea,West Brom,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.0,1.15,6.75,16.00,1.15,6.2,14.0,1.17,6.50,19.00,1.17,7.50,19.00
4,14/08/10,Sunderland,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.6,2.15,3.20,3.30,2.10,3.2,3.2,2.15,3.20,3.50,2.20,3.25,3.50


In [6]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match pathation.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0

    # Aggregate to get until that point
    for i in range(2, 39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

In [7]:
# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys

    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match pathation.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']

        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

In [8]:
def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)

    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC

    return playing_stat

In [9]:
# Apply to each dataset
# playing_statistics_1 = get_gss(playing_statistics_1)
# playing_statistics_2 = get_gss(playing_statistics_2)
# playing_statistics_3 = get_gss(playing_statistics_3)
# playing_statistics_4 = get_gss(playing_statistics_4)
# playing_statistics_5 = get_gss(playing_statistics_5)
# playing_statistics_6 = get_gss(playing_statistics_6)
# playing_statistics_7 = get_gss(playing_statistics_7)
# playing_statistics_8 = get_gss(playing_statistics_8)
# playing_statistics_9 = get_gss(playing_statistics_9)
# playing_statistics_10 = get_gss(playing_statistics_10)
playing_statistics_11 = get_gss(playing_statistics_11)
playing_statistics_12 = get_gss(playing_statistics_12)
playing_statistics_13 = get_gss(playing_statistics_13)
playing_statistics_14 = get_gss(playing_statistics_14)
playing_statistics_15 = get_gss(playing_statistics_15)
playing_statistics_16 = get_gss(playing_statistics_16)
playing_statistics_17 = get_gss(playing_statistics_17)
playing_statistics_18 = get_gss(playing_statistics_18)
playing_statistics_19 = get_gss(playing_statistics_19)
playing_statistics_20 = get_gss(playing_statistics_20)

In [10]:
# examples - when season 19/20 started

playing_statistics_20.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC
0,09/08/2019,Liverpool,Norwich,4,1,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.12,8.5,21.00,1.14,9.5,23.00,0,0,0,0
1,10/08/2019,West Ham,Man City,0,5,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,13.00,6.0,1.24,12.00,6.5,1.25,0,0,0,0
2,10/08/2019,Bournemouth,Sheffield United,1,1,D,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,1.95,3.60,3.90,1.97,3.55,3.80,2.00,3.5,3.80,2.00,3.6,4.00,0,0,0,0
3,10/08/2019,Burnley,Southampton,3,0,H,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.70,3.2,2.75,2.70,3.3,2.80,0,0,0,0
4,10/08/2019,Crystal Palace,Everton,0,0,D,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.20,3.20,2.35,3.10,3.20,2.40,3.10,3.3,2.35,3.20,3.3,2.45,0,0,0,0


In [11]:
# when it ended
playing_statistics_20.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC
375,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36
376,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70
377,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32
378,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36
379,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66


In [12]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0


def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]

    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points


def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')

    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_agg_points(playing_stat):

    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

# Apply to each dataset
# playing_statistics_1 = get_agg_points(playing_statistics_1)
# playing_statistics_2 = get_agg_points(playing_statistics_2)
# playing_statistics_3 = get_agg_points(playing_statistics_3)
# playing_statistics_4 = get_agg_points(playing_statistics_4)
# playing_statistics_5 = get_agg_points(playing_statistics_5)
# playing_statistics_6 = get_agg_points(playing_statistics_6)
# playing_statistics_7 = get_agg_points(playing_statistics_7)
# playing_statistics_8 = get_agg_points(playing_statistics_8)
# playing_statistics_9 = get_agg_points(playing_statistics_9)
# playing_statistics_10 = get_agg_points(playing_statistics_10)
playing_statistics_11 = get_agg_points(playing_statistics_11)
playing_statistics_12 = get_agg_points(playing_statistics_12)
playing_statistics_13 = get_agg_points(playing_statistics_13)
playing_statistics_14 = get_agg_points(playing_statistics_14)
playing_statistics_15 = get_agg_points(playing_statistics_15)
playing_statistics_16 = get_agg_points(playing_statistics_16)

playing_statistics_17 = get_agg_points(playing_statistics_17)
playing_statistics_18 = get_agg_points(playing_statistics_18)
playing_statistics_19 = get_agg_points(playing_statistics_19)
playing_statistics_20 = get_agg_points(playing_statistics_20)

In [13]:
playing_statistics_20.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP
0,09/08/2019,Liverpool,Norwich,4,1,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.12,8.5,21.00,1.14,9.5,23.00,0,0,0,0,0,0
1,10/08/2019,West Ham,Man City,0,5,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,13.00,6.0,1.24,12.00,6.5,1.25,0,0,0,0,0,0
2,10/08/2019,Bournemouth,Sheffield United,1,1,D,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,1.95,3.60,3.90,1.97,3.55,3.80,2.00,3.5,3.80,2.00,3.6,4.00,0,0,0,0,0,0
3,10/08/2019,Burnley,Southampton,3,0,H,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.70,3.2,2.75,2.70,3.3,2.80,0,0,0,0,0,0
4,10/08/2019,Crystal Palace,Everton,0,0,D,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.20,3.20,2.35,3.10,3.20,2.40,3.10,3.3,2.35,3.20,3.3,2.45,0,0,0,0,0,0


In [14]:
playing_statistics_20.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP
375,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36,62,63
376,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70,78,21
377,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32,44,96
378,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36,49,54
379,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66,38,34


In [15]:
def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]

    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam

        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent

        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent

        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h
    playing_stat['AM' + str(num)] = a


    return playing_stat

def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics

# Make changes to df
# playing_statistics_1 = add_form_df(playing_statistics_1)
# playing_statistics_2 = add_form_df(playing_statistics_2)
# playing_statistics_3 = add_form_df(playing_statistics_3)
# playing_statistics_4 = add_form_df(playing_statistics_4)
# playing_statistics_5 = add_form_df(playing_statistics_5)
# playing_statistics_6 = add_form_df(playing_statistics_6)
# playing_statistics_7 = add_form_df(playing_statistics_7)
# playing_statistics_8 = add_form_df(playing_statistics_8)
# playing_statistics_9 = add_form_df(playing_statistics_9)
# playing_statistics_10 = add_form_df(playing_statistics_10)
playing_statistics_11 = add_form_df(playing_statistics_11)
playing_statistics_12 = add_form_df(playing_statistics_12)
playing_statistics_13 = add_form_df(playing_statistics_13)
playing_statistics_14 = add_form_df(playing_statistics_14)
playing_statistics_15 = add_form_df(playing_statistics_15)
playing_statistics_16 = add_form_df(playing_statistics_16)
playing_statistics_17 = add_form_df(playing_statistics_17)
playing_statistics_18 = add_form_df(playing_statistics_18)
playing_statistics_19 = add_form_df(playing_statistics_19)
playing_statistics_20 = add_form_df(playing_statistics_20)

In [16]:
playing_statistics_20.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
0,09/08/2019,Liverpool,Norwich,4,1,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.12,8.5,21.00,1.14,9.5,23.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M
1,10/08/2019,West Ham,Man City,0,5,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,13.00,6.0,1.24,12.00,6.5,1.25,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M
2,10/08/2019,Bournemouth,Sheffield United,1,1,D,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,1.95,3.60,3.90,1.97,3.55,3.80,2.00,3.5,3.80,2.00,3.6,4.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M
3,10/08/2019,Burnley,Southampton,3,0,H,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.70,3.2,2.75,2.70,3.3,2.80,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M
4,10/08/2019,Crystal Palace,Everton,0,0,D,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.20,3.20,2.35,3.10,3.20,2.40,3.10,3.3,2.35,3.20,3.3,2.45,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M


In [17]:
playing_statistics_20.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
375,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36,62,63,L,D,W,W,L,D,D,W,W,W
376,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70,78,21,W,L,W,L,W,L,W,L,L,L
377,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32,44,96,D,W,L,L,L,D,L,W,D,W
378,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36,49,54,W,L,D,L,D,W,D,W,W,D
379,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66,38,34,D,W,W,D,W,W,L,L,D,L


In [18]:
Standings = pd.read_csv("/content/gdrive/MyDrive/csv_files/EPLStandings1.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.drop(columns = ['Unnamed: 0'])
#Standings = Standings.fillna(18)

def get_last(playing_stat, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    return playing_stat

# playing_statistics_1 = get_last(playing_statistics_1, Standings, 0)
# playing_statistics_2 = get_last(playing_statistics_2, Standings, 1)
# playing_statistics_3 = get_last(playing_statistics_3, Standings, 2)
# playing_statistics_4 = get_last(playing_statistics_4, Standings, 3)
# playing_statistics_5 = get_last(playing_statistics_5, Standings, 4)
# playing_statistics_6 = get_last(playing_statistics_6, Standings, 5)
# playing_statistics_7 = get_last(playing_statistics_7, Standings, 6)
# playing_statistics_8 = get_last(playing_statistics_8, Standings, 7)
# playing_statistics_9 = get_last(playing_statistics_9, Standings, 8)
# playing_statistics_10 = get_last(playing_statistics_10, Standings, 9)
playing_statistics_11 = get_last(playing_statistics_11, Standings, 10)
playing_statistics_12 = get_last(playing_statistics_12, Standings, 11)
playing_statistics_13 = get_last(playing_statistics_13, Standings, 12)
playing_statistics_14 = get_last(playing_statistics_14, Standings, 13)
playing_statistics_15 = get_last(playing_statistics_15, Standings, 14)
playing_statistics_16 = get_last(playing_statistics_16, Standings, 15)
playing_statistics_17 = get_last(playing_statistics_17, Standings, 16)
playing_statistics_18 = get_last(playing_statistics_18, Standings, 17)
playing_statistics_19 = get_last(playing_statistics_19, Standings, 18)
playing_statistics_20 = get_last(playing_statistics_20, Standings, 19)
#playing_statistics_21 = get_last(playing_statistics_21, Standings, 20)

In [19]:
columns = np.array(Standings.columns)

for i in range(len(columns)):
  year_array = np.array(Standings[columns[i]])
  indexes = np.argwhere(year_array == float(0))
  year_array[indexes] = 18
  Standings[columns[i]] = year_array

 

In [20]:
Standings.head(5)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Team,,,,,,,,,,,,,,,,,,,,,
Arsenal,2.0,2.0,1.0,2.0,1.0,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,2.0,5.0,6.0,5.0,8.0
Aston Villa,6.0,8.0,8.0,16.0,6.0,10.0,16.0,11.0,6.0,6.0,6.0,9.0,16.0,15.0,15.0,17.0,20.0,18.0,18.0,18.0,17.0
Birmingham,18.0,18.0,18.0,13.0,10.0,12.0,18.0,18.0,19.0,18.0,9.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackburn,18.0,18.0,10.0,6.0,15.0,15.0,6.0,10.0,7.0,15.0,10.0,15.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackpool,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0


In [21]:
Standings.shape

(45, 21)

In [22]:
Standings.head(5)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Team,,,,,,,,,,,,,,,,,,,,,
Arsenal,2.0,2.0,1.0,2.0,1.0,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,2.0,5.0,6.0,5.0,8.0
Aston Villa,6.0,8.0,8.0,16.0,6.0,10.0,16.0,11.0,6.0,6.0,6.0,9.0,16.0,15.0,15.0,17.0,20.0,18.0,18.0,18.0,17.0
Birmingham,18.0,18.0,18.0,13.0,10.0,12.0,18.0,18.0,19.0,18.0,9.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackburn,18.0,18.0,10.0,6.0,15.0,15.0,6.0,10.0,7.0,15.0,10.0,15.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackpool,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0


In [23]:
Standings.shape

(45, 21)

In [24]:
playing_statistics_20.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP
0,09/08/2019,Liverpool,Norwich,4,1,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.12,8.5,21.00,1.14,9.5,23.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,2.0,0.0
1,10/08/2019,West Ham,Man City,0,5,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,13.00,6.0,1.24,12.00,6.5,1.25,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,10.0,1.0
2,10/08/2019,Bournemouth,Sheffield United,1,1,D,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,1.95,3.60,3.90,1.97,3.55,3.80,2.00,3.5,3.80,2.00,3.6,4.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,14.0,0.0
3,10/08/2019,Burnley,Southampton,3,0,H,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.70,3.2,2.75,2.70,3.3,2.80,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,15.0,16.0
4,10/08/2019,Crystal Palace,Everton,0,0,D,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.20,3.20,2.35,3.10,3.20,2.40,3.10,3.3,2.35,3.20,3.3,2.45,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,12.0,8.0


In [25]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

# playing_statistics_1 = get_mw(playing_statistics_1)
# playing_statistics_2 = get_mw(playing_statistics_2)
# playing_statistics_3 = get_mw(playing_statistics_3)
# playing_statistics_4 = get_mw(playing_statistics_4)
# playing_statistics_5 = get_mw(playing_statistics_5)
# playing_statistics_6 = get_mw(playing_statistics_6)
# playing_statistics_7 = get_mw(playing_statistics_7)
# playing_statistics_8 = get_mw(playing_statistics_8)
# playing_statistics_9 = get_mw(playing_statistics_9)
# playing_statistics_10 = get_mw(playing_statistics_10)
playing_statistics_11 = get_mw(playing_statistics_11)
playing_statistics_12 = get_mw(playing_statistics_12)
playing_statistics_13 = get_mw(playing_statistics_13)
playing_statistics_14 = get_mw(playing_statistics_14)
playing_statistics_15 = get_mw(playing_statistics_15)
playing_statistics_16 = get_mw(playing_statistics_16)
playing_statistics_17 = get_mw(playing_statistics_17)
playing_statistics_18 = get_mw(playing_statistics_18)
playing_statistics_19 = get_mw(playing_statistics_19)
playing_statistics_20 = get_mw(playing_statistics_20)

In [26]:
playing_statistics_20.head(15)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW
0,09/08/2019,Liverpool,Norwich,4,1,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.12,8.50,21.00,1.14,9.5,23.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,2.0,0.0,1
1,10/08/2019,West Ham,Man City,0,5,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,13.00,6.00,1.24,12.00,6.5,1.25,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,10.0,1.0,1
2,10/08/2019,Bournemouth,Sheffield United,1,1,D,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,1.95,3.60,3.90,1.97,3.55,3.80,2.00,3.50,3.80,2.00,3.6,4.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,14.0,0.0,1
3,10/08/2019,Burnley,Southampton,3,0,H,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.70,3.20,2.75,2.70,3.3,2.80,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,15.0,16.0,1
4,10/08/2019,Crystal Palace,Everton,0,0,D,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.20,3.20,2.35,3.10,3.20,2.40,3.10,3.30,2.35,3.20,3.3,2.45,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,12.0,8.0,1
5,10/08/2019,Watford,Brighton,0,3,A,11,5,3,3,15,11,5,2,0,1,0,0,1.90,3.40,4.00,1.90,3.40,4.33,1.93,3.40,4.25,1.95,3.40,4.20,1.95,3.5,4.33,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,11.0,17.0,1
6,10/08/2019,Tottenham,Aston Villa,3,1,H,31,7,7,4,13,9,14,0,1,0,0,0,1.30,5.25,10.00,1.30,5.50,10.00,1.30,5.50,9.60,1.29,5.50,10.00,1.30,5.5,12.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,4.0,0.0,1
7,11/08/2019,Leicester,Wolves,0,0,D,15,8,1,2,3,13,12,3,0,2,0,0,2.20,3.20,3.40,2.25,3.30,3.30,2.20,3.25,3.45,2.20,3.25,3.50,2.25,3.3,3.60,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,9.0,7.0,1
8,11/08/2019,Newcastle,Arsenal,0,1,A,9,8,2,2,12,7,5,3,1,3,0,0,4.50,3.75,1.72,4.50,3.75,1.78,4.40,3.85,1.77,4.50,3.75,1.78,4.60,3.9,1.80,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,13.0,5.0,1
9,11/08/2019,Man United,Chelsea,4,0,H,11,18,5,7,15,13,3,5,3,4,0,0,2.10,3.30,3.50,2.15,3.30,3.50,2.15,3.35,3.40,2.15,3.30,3.50,2.25,3.3,3.50,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,6.0,3.0,1


In [27]:
  # playing_statistics_1,
  #                           playing_statistics_2,
  #                           playing_statistics_3,
  #                           playing_statistics_4,
  #                           playing_statistics_5,
  #                           playing_statistics_6,
  #                           playing_statistics_7,
  #                           playing_statistics_8,
  #                           playing_statistics_9,
  #                           playing_statistics_10,

playing_stat = pd.concat([
                          playing_statistics_11,
                          playing_statistics_12,
                          playing_statistics_13,
                          playing_statistics_14,
                          playing_statistics_15,
                          playing_statistics_16,
                          playing_statistics_17,
                          playing_statistics_18,
                          playing_statistics_19,
                          playing_statistics_20], ignore_index=True)

In [28]:
playing_stat[playing_stat.duplicated()]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW


In [29]:
playing_stat.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW
0,14/08/10,Aston Villa,West Ham,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1
1,14/08/10,Blackburn,Everton,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1
2,14/08/10,Bolton,Fulham,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.4,2.15,3.25,3.25,2.20,3.2,3.0,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1
3,14/08/10,Chelsea,West Brom,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.0,1.15,6.75,16.00,1.15,6.2,14.0,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1
4,14/08/10,Sunderland,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.6,2.15,3.20,3.30,2.10,3.2,3.2,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1


In [30]:
playing_stat.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW
3795,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36,62,63,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38
3796,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70,78,21,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38
3797,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32,44,96,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38
3798,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36,49,54,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38
3799,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66,38,34,D,W,W,D,W,W,L,L,D,L,10.0,0.0,38


In [31]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

playing_stat['HTFormPtsStr'] = playing_stat['HM1'] + playing_stat['HM2'] + playing_stat['HM3'] + playing_stat['HM4'] + playing_stat['HM5']
playing_stat['ATFormPtsStr'] = playing_stat['AM1'] + playing_stat['AM2'] + playing_stat['AM3'] + playing_stat['AM4'] + playing_stat['AM5']

playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)

In [32]:
playing_stat.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts
0,14/08/10,Aston Villa,West Ham,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1,MMMMM,MMMMM,0,0
1,14/08/10,Blackburn,Everton,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1,MMMMM,MMMMM,0,0
2,14/08/10,Bolton,Fulham,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.4,2.15,3.25,3.25,2.20,3.2,3.0,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1,MMMMM,MMMMM,0,0
3,14/08/10,Chelsea,West Brom,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.0,1.15,6.75,16.00,1.15,6.2,14.0,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1,MMMMM,MMMMM,0,0
4,14/08/10,Sunderland,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.6,2.15,3.20,3.30,2.10,3.2,3.2,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1,MMMMM,MMMMM,0,0


In [33]:
playing_stat.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts
3795,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36,62,63,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38,LWLDW,DWDWW,7,11
3796,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70,78,21,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38,WWWWL,LLLLL,12,0
3797,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32,44,96,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38,DLLLD,WLDWW,2,10
3798,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36,49,54,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38,WDDDW,LLWWD,9,7
3799,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66,38,34,D,W,W,D,W,W,L,L,D,L,10.0,0.0,38,DWWLD,WDWLL,8,7


In [34]:
# Get Goal Difference
playing_stat['HTGD'] = playing_stat['HTGS'] - playing_stat['HTGC']
playing_stat['ATGD'] = playing_stat['ATGS'] - playing_stat['ATGC']

# Diff in points
playing_stat['DiffPts'] = playing_stat['HTP'] - playing_stat['ATP']
playing_stat['DiffFormPts'] = playing_stat['HTFormPts'] - playing_stat['ATFormPts']

# Diff in last year positions
playing_stat['DiffLP'] = playing_stat['HomeTeamLP'] - playing_stat['AwayTeamLP']

In [35]:
playing_stat.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,14/08/10,Aston Villa,West Ham,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1,MMMMM,MMMMM,0,0,0,0,0,0,-11.0
1,14/08/10,Blackburn,Everton,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1,MMMMM,MMMMM,0,0,0,0,0,0,2.0
2,14/08/10,Bolton,Fulham,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.4,2.15,3.25,3.25,2.20,3.2,3.0,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1,MMMMM,MMMMM,0,0,0,0,0,0,2.0
3,14/08/10,Chelsea,West Brom,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.0,1.15,6.75,16.00,1.15,6.2,14.0,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1,MMMMM,MMMMM,0,0,0,0,0,0,1.0
4,14/08/10,Sunderland,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.6,2.15,3.20,3.30,2.10,3.2,3.2,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1,MMMMM,MMMMM,0,0,0,0,0,0,4.0


In [36]:
playing_stat.tail(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
3795,26/07/2020,Leicester,Man United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.4,2.25,3.20,3.4,2.25,67,64,39,36,62,63,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38,LWLDW,DWDWW,7,11,28,28,-1,-4,3.0
3796,26/07/2020,Man City,Norwich,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.0,21.00,1.05,13.0,34.00,97,26,35,70,78,21,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38,WWWWL,LLLLL,12,0,62,-44,57,12,1.0
3797,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.0,1.38,8.00,4.8,1.40,37,82,55,32,44,96,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38,DLLLD,WLDWW,2,10,-18,50,-52,-8,11.0
3798,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.4,3.40,2.15,3.4,3.40,48,38,59,36,49,54,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38,WDDDW,LLWWD,9,7,-11,2,-5,2,16.0
3799,26/07/2020,West Ham,Aston Villa,1,1,D,10,13,1,4,16,13,0,7,2,1,0,0,3.00,3.60,2.30,3.00,3.50,2.30,3.05,3.40,2.30,3.00,3.6,2.30,3.10,3.6,2.25,48,40,61,66,38,34,D,W,W,D,W,W,L,L,D,L,10.0,0.0,38,DWWLD,WDWLL,8,7,-13,-26,4,1,10.0


In [37]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
playing_stat.MW = playing_stat.MW.astype(float)

for col in cols:
    playing_stat[col] = playing_stat[col] / playing_stat.MW

In [38]:
playing_stat.head(15)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,14/08/10,Aston Villa,West Ham,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.4,4.0,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,-11.0
1,14/08/10,Blackburn,Everton,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.2,2.5,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,2.0
2,14/08/10,Bolton,Fulham,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.40,2.15,3.25,3.25,2.20,3.2,3.0,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,2.0
3,14/08/10,Chelsea,West Brom,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.00,1.15,6.75,16.00,1.15,6.2,14.0,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,1.0
4,14/08/10,Sunderland,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.60,2.15,3.20,3.30,2.10,3.2,3.2,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,4.0
5,14/08/10,Tottenham,Man City,0,0,D,22,11,18,7,13,16,10,3,0,2,0,0,2.40,3.30,3.00,2.35,3.30,2.85,2.40,3.2,2.7,2.38,3.25,3.00,2.30,3.40,3.10,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,4.0,5.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,-1.0
6,14/08/10,Wigan,Blackpool,0,4,A,11,9,6,7,8,11,6,4,1,1,0,0,1.83,3.50,4.50,1.78,3.40,4.50,1.90,3.3,3.7,1.75,3.40,5.00,1.83,3.40,4.75,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,16.0,0.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,16.0
7,14/08/10,Wolves,Stoke,2,1,H,13,10,7,6,17,13,5,5,0,2,0,0,2.30,3.25,3.20,2.25,3.15,3.10,2.30,3.2,2.8,2.30,3.20,3.20,2.30,3.25,3.30,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,15.0,11.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,4.0
8,15/08/10,Liverpool,Arsenal,1,1,D,7,14,4,7,13,15,9,11,1,3,1,1,2.50,3.25,2.88,2.45,3.25,2.75,2.40,3.2,2.7,2.50,3.20,2.88,2.50,3.20,3.00,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,7.0,3.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,4.0
9,16/08/10,Man United,Newcastle,3,0,H,18,7,10,3,9,5,5,3,2,2,0,0,1.25,5.50,15.00,1.22,5.50,12.50,1.22,5.3,11.0,1.22,6.00,13.00,1.22,6.00,15.00,0,0,0,0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,2.0,0.0,1.0,MMMMM,MMMMM,0,0,0.0,0.0,0.0,0.0,2.0


In [39]:
playing_stat.to_csv('finaldataset_bookmakers.csv', sep = ',', index = False)

# Add news data in historic data and generate final dataframe

## Pick data from google drive 

In [40]:
# pick data from google drive 
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/Noticias_Premier_League_all_teams/"
path2= "/content/gdrive/MyDrive/csv_files/finaldataset_bookmakers.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Create functions to use in DataFrame


In [41]:
# Create function to change de format of date
def change_data(v,x,d):
  if len(v[d][x])==8:
    year = '2'+'0'+v[d][x][6]+v[d][x][7]
    month = v[d][x][3]+ v[d][x][4]
    day = v[d][x][0]+ v[d][x][1]
    date = year+'-'+month+'-'+day
    return date
  elif len(v[d][x])==10:
      year = v[d][x][6]+v[d][x][7]+v[d][x][8]+v[d][x][9]
      month = v[d][x][3]+ v[d][x][4]
      day = v[d][x][0]+ v[d][x][1]
      date = year+'-'+month+'-'+day
      return date


In [42]:
# Create function to apply dummies
def dummies(resultado_d,x):
  if x == 'H':
    resultado_d.append(1)
  elif x=='D':
    resultado_d.append(2)
  elif x=='A':
    resultado_d.append(3)
    

In [43]:
# Create a function to pick news ten days ago the game 
def get_news(dataframe,x,timev,casa):
  time = dataframe.loc[x,casa]                    
  vetor_resultado = []
  if time in timev:
    vetor_noticias=[]
    arquivo_noticias = times[time]
    data_atual = date.fromisoformat(dataframe['Date'][x])
    for y in range (1,10):
      data_analisada = data_atual - timedelta(days=y)
      data = data_analisada.isoformat()
      n_atual =arquivo_noticias.loc[arquivo_noticias['date']==data]
      vetor = np.array(n_atual)
      for k in range (0,len(vetor)):
          texto = ''
          for h in range(3,6):
            texto = texto+''+str(vetor[k][h])
          vetor_noticias.append(texto)
    data_frame_vetor = pd.DataFrame(vetor_noticias)
    data_frame_vetor.drop_duplicates(inplace = True)
    vetor_resultado = np.array(data_frame_vetor)
  return vetor_resultado

# Importing libraries

In [44]:
#import libraries
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV 
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date, timedelta
import pandas as pd
import seaborn as sns
from google.colab import drive
! pip install pattern
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pattern
from pattern.en import lemma, lexeme

In [45]:
# pick data from google drive 
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/csv_files/Data_Doc2Vec/"
path2= "/content/gdrive/MyDrive/csv_files/finaldataset_bookmakers.csv"

#path = '/content/gdrive/MyDrive/Noticias_Premier_League'
#path2 = '/content/gdrive/MyDrive/csv_files/finaldataset.csv'



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Preparing Data

## Create functions to use in DataFrame


In [46]:
# Create function to change de format of date
def change_data(v,x,d):
  if len(v[d][x])==8:
    year = '2'+'0'+v[d][x][6]+v[d][x][7]
    month = v[d][x][3]+ v[d][x][4]
    day = v[d][x][0]+ v[d][x][1]
    date = year+'-'+month+'-'+day
    return date
  elif len(v[d][x])==10:
      year = v[d][x][6]+v[d][x][7]+v[d][x][8]+v[d][x][9]
      month = v[d][x][3]+ v[d][x][4]
      day = v[d][x][0]+ v[d][x][1]
      date = year+'-'+month+'-'+day
      return date


In [47]:
# Create function to apply dummies
def dummies(resultado_d,x):
  if x == 'H':
    resultado_d.append(1)
  elif x=='D':
    resultado_d.append(2)
  elif x=='A':
    resultado_d.append(3)
    

In [48]:
# Create a function to pick news ten days ago the game 
def get_news(dataframe,x,timev,casa):
  time = dataframe.loc[x,casa]                    
  vetor_resultado = []
  if time in timev:
    vetor_noticias=[]
    arquivo_noticias = times[time]
    data_atual = date.fromisoformat(dataframe['Date'][x])
    for y in range (1,10):
      data_analisada = data_atual - timedelta(days=y)
      data = data_analisada.isoformat()
      n_atual =arquivo_noticias.loc[arquivo_noticias['date']==data]
      vetor = np.array(n_atual)
      for k in range (0,len(vetor)):
          texto = ''
          for h in range(3,6):
            texto = texto+''+str(vetor[k][h])
          vetor_noticias.append(texto)
    data_frame_vetor = pd.DataFrame(vetor_noticias)
    data_frame_vetor.drop_duplicates(inplace = True)
    vetor_resultado = np.array(data_frame_vetor)
  return vetor_resultado

##Create dataframes using created fuctions in data


In [49]:
# Create dataframe with date of games and replace abbreviations of teams
dfp = pd.read_csv(path2)
dfp.replace('Man United','Manchester United',inplace=True)
dfp.replace ('Man City','Manchester City',inplace=True)
dfp.replace ('West Ham','West Ham United',inplace=True)
dfp.replace ('Chelsea','Chelsea FC',inplace=True)
dfp.replace ('Liverpool','Liverpool FC',inplace=True)
dfp.replace ('Fulham','Fulham FC',inplace=True)
dfp.replace ('Burnley','Burnley FC',inplace=True)
dfp.replace ('Southampton','Southampton FC',inplace=True)
dfp.replace ('Everton','Everton FC',inplace=True)
dfp.replace ('Arsenal','Arsenal FC',inplace=True)
dfp.replace ('Newcastle','Newcastle United',inplace=True)
dfp.replace ('West Brom','West Bromwich Albion',inplace=True)
dfp.replace ('Wolves','Wolverhampton Wanderers',inplace=True)
dfp.replace ('Tottenham','Tottenham Hotspur',inplace=True)
dfp.replace ('Norwich','Norwich City',inplace=True)
dfp.replace ('QPR','Queens Park Rangers',inplace=True)
dfp.replace ('Reading','Reading FC',inplace=True)
dfp.replace ('Stoke','Stoke City',inplace=True)
dfp.replace ('Sunderland','Sunderland AFC',inplace=True)
dfp.replace ('Leicester','Leicester City',inplace=True)
dfp.replace ('Bournemouth','AFC Bournemouth',inplace=True)
dfp.replace ('Middlesbrough','Middlesbrough FC',inplace=True)
dfp.replace ('Brighton','Brighton & Hove Albion',inplace=True)
dfp.replace ('Huddersfield','Huddersfield Town',inplace=True)

dfp


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,14/08/10,Aston Villa,West Ham United,3,0,H,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.40,4.00,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-11.0
1,14/08/10,Blackburn,Everton FC,1,0,H,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.20,2.50,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0
2,14/08/10,Bolton,Fulham FC,0,0,D,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.40,2.15,3.25,3.25,2.20,3.20,3.00,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0
3,14/08/10,Chelsea FC,West Bromwich Albion,6,0,H,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.00,1.15,6.75,16.00,1.15,6.20,14.00,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,1.0
4,14/08/10,Sunderland AFC,Birmingham,2,2,D,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.60,2.15,3.20,3.30,2.10,3.20,3.20,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,26/07/2020,Leicester City,Manchester United,0,2,A,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.40,2.25,3.20,3.40,2.25,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0
3796,26/07/2020,Manchester City,Norwich City,5,0,H,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.00,21.00,1.05,13.00,34.00,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0
3797,26/07/2020,Newcastle United,Liverpool FC,1,3,A,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.00,1.38,8.00,4.80,1.40,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0
3798,26/07/2020,Southampton FC,Sheffield United,3,1,H,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.40,3.40,2.15,3.40,3.40,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0


In [50]:
# apply change data in dfp
for x in range (0,len(dfp)):
  dfp.loc[x,'Date']=change_data(dfp,x,'Date')

In [51]:
# Separate target
resultado = dfp['FTR']

In [52]:
# apply dummies in results
resultado_d = []
for y in resultado:
  dummies(resultado_d,y)

In [53]:
# Create Features Noticias Home and Noticias Away
noticias_home = []
noticias_away = []
for x in range(0,len(resultado_d)):
  noticias_home.append([])
  noticias_away.append([])

In [54]:
# Aplly features in DataFrame
dfp['Noticias Home']=noticias_home
dfp['Noticias Away']=noticias_away
dfp['FTR'] = resultado_d

In [55]:
dfp

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away
0,2010-08-14,Aston Villa,West Ham United,3,0,1,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.40,4.00,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-11.0,[],[]
1,2010-08-14,Blackburn,Everton FC,1,0,1,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.20,2.50,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0,[],[]
2,2010-08-14,Bolton,Fulham FC,0,0,2,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.40,2.15,3.25,3.25,2.20,3.20,3.00,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0,[],[]
3,2010-08-14,Chelsea FC,West Bromwich Albion,6,0,1,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.00,1.15,6.75,16.00,1.15,6.20,14.00,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,1.0,[],[]
4,2010-08-14,Sunderland AFC,Birmingham,2,2,2,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.60,2.15,3.20,3.30,2.10,3.20,3.20,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,4.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2020-07-26,Leicester City,Manchester United,0,2,3,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.40,2.25,3.20,3.40,2.25,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0,[],[]
3796,2020-07-26,Manchester City,Norwich City,5,0,1,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.00,21.00,1.05,13.00,34.00,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0,[],[]
3797,2020-07-26,Newcastle United,Liverpool FC,1,3,3,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.00,1.38,8.00,4.80,1.40,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0,[],[]
3798,2020-07-26,Southampton FC,Sheffield United,3,1,1,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.40,3.40,2.15,3.40,3.40,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0,[],[]


In [56]:
# Create dataframes news of teams
team0=pd.read_csv(path + 'Arsenal FC.csv' )
team1=pd.read_csv(path + 'Aston Villa.csv' )
team2=pd.read_csv(path + 'Brighton _ Hove Albion.csv' )
team3=pd.read_csv(path + 'Burnley FC.csv' )
team4=pd.read_csv(path + 'Chelsea FC.csv' )
team5=pd.read_csv(path + 'Crystal Palace.csv' )
team6=pd.read_csv(path + 'Everton FC.csv' )
team7=pd.read_csv(path + 'Fulham FC.csv' )
team8=pd.read_csv(path + 'Leeds United.csv' )
team9=pd.read_csv(path + 'Leicester City.csv' )
team10=pd.read_csv(path + 'Liverpool FC.csv' )
team11=pd.read_csv(path + 'Manchester City.csv' )
team12=pd.read_csv(path + 'Manchester United.csv' )
team13=pd.read_csv(path + 'Newcastle United.csv' )
team14=pd.read_csv(path + 'Sheffield United.csv' )
team15=pd.read_csv(path + 'Southampton FC.csv' )
team16=pd.read_csv(path + 'Tottenham Hotspur.csv' )
team17=pd.read_csv(path + 'West Bromwich Albion.csv' )
team18=pd.read_csv(path + 'West Ham United.csv' )
team19=pd.read_csv(path + 'Wolverhampton Wanderers.csv')
team20=pd.read_csv(path +'Cardiff City.csv')
team21=pd.read_csv(path +'Hull City.csv')
team22=pd.read_csv(path +'Swansea City.csv')
team23=pd.read_csv(path +'Wigan Athletic.csv')
team24=pd.read_csv(path +'Norwich City.csv')
team25=pd.read_csv(path +'Queens Park Rangers.csv')
team26=pd.read_csv(path +'Stoke City.csv')
team27=pd.read_csv(path +'Sunderland AFC.csv')
team28=pd.read_csv(path +'AFC Bournemouth.csv')
team29=pd.read_csv(path +'Watford FC.csv')
team30=pd.read_csv(path +'Middlesbrough FC.csv')
team31=pd.read_csv(path +'Huddersfield Town.csv')



In [57]:
# Create diconary relating teams with yours dataframes 
times = {'Arsenal FC':team0,'Aston Villa':team1,'Brighton & Hove Albion':team2,'Burnley FC':team3,'Chelsea FC':team4, 'Crystal Palace':team5,'Everton FC':team6,'Fulham FC':team7,'Leeds United':team8,'Leicester City':team9,'Liverpool FC':team10,'Manchester City':team11,'Manchester United':team12,'Newcastle United':team13,'Sheffield United':team14,'Southampton FC':team15,'Tottenham Hotspur':team16,'West Bromwich Albion':team17,'West Ham United':team18,'Wolverhampton Wanderers':team19
         ,'Cardiff City':team20,'Hull City':team21,'Swansea City':team22,'Wigan Athletic':team23,'Norwich City':team24,'Queens Park Rangers':team25,'Stoke City':team26,
         'Sunderland AFC':team27,'AFC Bournemouth':team28,'Watford FC':team29,'Middlesbrough FC':team30,'Huddersfield Town':team31}
times_vetor = list(times.keys())

In [58]:
times['Arsenal FC']

,Unnamed: 0,index,date,hour,title,subtitle,text
0,0,0,2021-02-12,15:00,Arteta joins calls for social media change,Arsenal manager Mikel Arteta said Friday he ha...,The Spaniard said he no longer uses his Twitte...
1,1,1,2021-02-12,12:18,Arsenal v Benfica Europa League return leg swi...,Arsenal's home Europa League tie against Portu...,The game will be played at the Georgios Karais...
2,2,2,2021-02-07,23:38,Three talking points from the Premier League,Manchester City tightened their grip on the Pr...,Victory at Anfield was one of the few remainin...
3,3,3,2021-02-06,18:17,Watkins strikes early as Villa sink Arsenal,Mikel Arteta urged Arsenal to be more ruthless...,Arteta's side fell behind after just 74 second...
4,4,4,2021-02-02,21:17,Nine-man Arsenal beaten by Wolves,Arsenal's ill-discipline cost Mikel Arteta's m...,A seven-game unbeaten league run had propelled...
...,...,...,...,...,...,...,...
14873,14873,14873,2010-07-20,11:43 am,talkSPORT Exclusive – Hodgson: ‘Gerrard was ke...,\n\tRoy Hodgson has told talkSPORT that Steven...,\n\tThe new Anfield boss pulled off a major co...
14874,14874,14874,2010-07-18,12:19 pm,George Graham: ‘For Capello to come out with t...,\n\tEx-Arsenal and Tottenham manager George Gr...,NaN
14875,14875,14875,2010-07-12,10:49 am,talkSPORT Exclusive – Redknapp: ‘I want Joe Cole’,\n\tTottenham boss Harry Redknapp has told tal...,\n\tThe England midfielder is a free agent aft...
14876,14876,14876,2010-04-01,11:00 pm,Clips of the Week – April 2,\n\tMike Parry reads out e-mails in morse code...,NaN


In [59]:
# Create a Dataframe with games as from 2013
df = dfp.loc[4758:]
df.reset_index(drop = True, inplace= True)
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away


In [60]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away


In [61]:
dfp

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away
0,2010-08-14,Aston Villa,West Ham United,3,0,1,23,12,11,2,15,15,16,7,1,2,0,0,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.40,4.00,1.91,3.25,4.33,2.00,3.25,4.20,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,6.0,17.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-11.0,[],[]
1,2010-08-14,Blackburn,Everton FC,1,0,1,7,17,2,12,19,14,1,3,2,1,0,0,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.20,2.50,2.88,3.20,2.50,3.10,3.30,2.38,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,10.0,8.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0,[],[]
2,2010-08-14,Bolton,Fulham FC,0,0,2,13,12,9,7,12,13,4,8,1,3,0,0,2.20,3.30,3.40,2.15,3.25,3.25,2.20,3.20,3.00,2.15,3.20,3.50,2.20,3.30,3.40,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0,[],[]
3,2010-08-14,Chelsea FC,West Bromwich Albion,6,0,1,18,10,13,4,10,10,3,1,1,0,0,0,1.17,7.00,17.00,1.15,6.75,16.00,1.15,6.20,14.00,1.17,6.50,19.00,1.17,7.50,19.00,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,1.0,0.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,1.0,[],[]
4,2010-08-14,Sunderland AFC,Birmingham,2,2,2,6,13,2,7,13,10,3,6,3,3,1,0,2.10,3.30,3.60,2.15,3.20,3.30,2.10,3.20,3.20,2.15,3.20,3.50,2.20,3.25,3.50,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,13.0,9.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,4.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2020-07-26,Leicester City,Manchester United,0,2,3,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.40,2.25,3.20,3.40,2.25,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0,[],[]
3796,2020-07-26,Manchester City,Norwich City,5,0,1,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.00,21.00,1.05,13.00,34.00,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0,[],[]
3797,2020-07-26,Newcastle United,Liverpool FC,1,3,3,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.00,1.38,8.00,4.80,1.40,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0,[],[]
3798,2020-07-26,Southampton FC,Sheffield United,3,1,1,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.40,3.40,2.15,3.40,3.40,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0,[],[]


In [62]:
# apply data change in news
for x in times_vetor:
  time = times[x]
  for y in range(0,len(time)):
    time.loc[y,'date']=change_data(time,y,'date')

In [63]:
#Add HomeTeam and AwayTeam news
for x in range (0,len(dfp)):
    dfp['Noticias Home'][x]=get_news(dfp,x,times_vetor,'HomeTeam')
    dfp['Noticias Away'][x]=get_news(dfp,x,times_vetor,'AwayTeam') 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
dfp[900:]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA,VCH,VCD,VCA,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away
900,2012-12-01,Fulham FC,Tottenham Hotspur,0,3,3,8,10,2,6,10,8,2,1,0,2,0,0,2.88,3.30,2.50,2.90,3.30,2.35,2.75,3.20,2.45,3.00,3.00,2.50,3.00,3.40,2.45,25,25,23,23,1.133333,1.533333,D,W,L,W,L,L,D,L,D,L,9.0,4.0,15.0,DLLDD,WWLLL,3,6,0.133333,0.133333,-0.400000,-0.200000,5.0,[],[]
901,2012-12-01,Liverpool FC,Southampton FC,1,0,1,24,8,11,4,10,8,10,4,2,0,0,0,1.40,4.50,8.50,1.40,4.50,7.50,1.40,4.40,7.30,1.44,4.00,8.00,1.44,4.75,8.50,18,21,18,31,1.066667,0.800000,L,D,D,W,W,W,D,D,D,L,8.0,0.0,15.0,LDWDD,DWWDL,6,8,0.000000,-0.666667,0.266667,-0.133333,8.0,[],[]
902,2012-12-01,Manchester City,Everton FC,1,1,2,9,8,6,5,16,12,5,2,2,1,0,0,1.50,4.00,7.00,1.55,3.70,6.50,1.50,4.00,6.10,1.57,3.80,6.00,1.55,4.33,6.50,27,24,10,18,2.133333,1.466667,W,D,D,D,W,L,W,W,D,D,1.0,7.0,15.0,WDWWD,DDLWD,11,6,1.133333,0.400000,0.666667,0.333333,-6.0,[],[]
903,2012-12-01,Queens Park Rangers,Aston Villa,1,1,2,17,7,12,3,5,6,11,4,1,2,0,0,1.95,3.50,4.00,1.95,3.40,3.80,2.00,3.30,3.60,2.05,3.25,3.75,2.00,3.50,4.00,10,11,26,22,0.333333,0.866667,D,W,L,D,L,L,L,L,D,W,17.0,16.0,15.0,DLLLD,WDLLW,2,7,-1.066667,-0.733333,-0.533333,-0.333333,1.0,[],[]
904,2012-12-01,Reading FC,Manchester United,3,4,3,10,10,7,5,10,12,6,1,1,1,0,0,7.50,4.75,1.40,7.75,4.40,1.40,7.20,3.80,1.40,8.00,4.00,1.44,8.00,4.80,1.44,19,33,27,18,0.600000,2.200000,L,W,L,W,L,L,W,W,D,W,0.0,2.0,15.0,LLLWD,WWLWW,4,12,-0.533333,1.000000,-1.600000,-0.533333,-2.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2020-07-26,Leicester City,Manchester United,0,2,3,14,7,3,3,12,11,3,3,1,4,1,0,3.25,3.50,2.20,3.20,3.50,2.20,3.15,3.25,2.30,3.20,3.40,2.25,3.20,3.40,2.25,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0,[],[]
3796,2020-07-26,Manchester City,Norwich City,5,0,1,31,5,10,4,7,4,9,0,1,1,0,0,1.08,11.00,26.00,1.05,13.50,41.00,1.07,12.00,30.00,1.07,15.00,21.00,1.05,13.00,34.00,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0,[],[]
3797,2020-07-26,Newcastle United,Liverpool FC,1,3,3,3,14,2,6,11,5,2,4,1,0,0,0,7.50,5.25,1.36,7.50,4.75,1.42,7.50,4.80,1.40,8.00,5.00,1.38,8.00,4.80,1.40,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0,[],[]
3798,2020-07-26,Southampton FC,Sheffield United,3,1,1,13,5,4,3,9,16,9,1,0,1,0,0,2.15,3.50,3.30,2.20,3.50,3.20,2.20,3.30,3.35,2.15,3.40,3.40,2.15,3.40,3.40,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0,[],[]
